#Workshop Title: "Hands-On Cybersecurity"
###Workshop Objectives
-Learn to secure systems using real tools and best practices.

-Implement authentication and encryption mechanisms.

-Explore passwordless authentication (Passkeys & TOTP).

-Configure and test firewalls and VPNs for secure networking.



### Task 1: Secure Password Storage Demo
🎯 Objective
Learn how to:

Hash passwords securely with salt

Store hash and salt

Validate passwords without storing the original password



**1.Environment Setup**
Install Python (version ≥ 3.6 recommended) and the bcrypt library.


In [ ]:
!pip install bcrypt


**2. Import Libraries**

In [ ]:
import bcrypt


**3. Password Hashing Process**
✅ Input a Password

In [ ]:
password = input("Enter your password: ").encode('utf-8')

**🔑 Generate a Salt**


In [ ]:
salt = bcrypt.gensalt()
print("Generated Salt:", salt)

🔐 **Hash the Password + Salt**

In [ ]:
hashed_password = bcrypt.hashpw(password, salt)
print("Stored Hash:", hashed_password)


**4. Simulate Storage (e.g., a database)**


In [ ]:
# Simulating a user record
user_db = {
    "username": "alice",
    "salt": salt,
    "hashed_password": hashed_password
}


**5. Password Validation Logic**


In [ ]:
def validate_password(stored_hash, input_password, salt):
    # Re-hash the input with stored salt
    input_hash = bcrypt.hashpw(input_password.encode('utf-8'), salt)
    return input_hash == stored_hash


**🔍 Test Validatio**n


In [ ]:
user_input = input("Enter your password again to validate: ")
is_valid = validate_password(user_db['hashed_password'], user_input, user_db['salt'])

if is_valid:
    print("✅ Password is correct!")
else:
    print("❌ Password is incorrect!")


🔒 **Security Notes**

Never store plaintext passwords.

Use strong hashing algorithms like bcrypt, scrypt, or argon2.

Always use a unique salt per user.

Store the salt and hash safely in your database.

### Password Hashing with scrypt and argon2

🔧**1. Prerequisites**
Install required libraries:

In [ ]:
!pip install argon2-cffi

No extra package is needed for scrypt, as it is available in the Python standard library (≥ 3.6).



### **Why scrypt?**
Resistant to GPU/ASIC cracking

Built into Python

Good for applications needing backward compatibility

🔐 **1. Hash a password**


In [ ]:
import os
import hashlib

# Input password
password = input("Enter your password: ").encode()

**🔑 Generate a Salt**


In [ ]:
# Generate salt
salt = os.urandom(16)

🔐 **Hash the Password + Salt**

In [ ]:
# Hash using scrypt
hashed = hashlib.scrypt(
    password,
    salt=salt,
    n=16384,
    r=8,
    p=1,
    dklen=64
)

print("Salt:", salt.hex())
print("Hash:", hashed.hex())




**Password Validation Logic**


In [ ]:
def validate_scrypt(input_password, stored_hash, salt):
    new_hash = hashlib.scrypt(
        input_password.encode(),
        salt=salt,
        n=16384,
        r=8,
        p=1,
        dklen=64
    )
    return new_hash == stored_hash

**🔍 Test Validatio**n


In [ ]:
# Validation test
user_input = input("Enter your password again: ")
is_valid = validate_scrypt(user_input, hashed, salt)

print("✅ Valid Password!" if is_valid else "❌ Invalid Password")

### Task 2: Simulating TOTP with Google Authenticator


**Prerequisites**
Install required libraries:

In [ ]:
!pip install pyotp qrcode[pil]

####Step 1: Generate a TOTP Secret


In [ ]:
import pyotp

# Generate a base32 secret
secret = pyotp.random_base32()
print(f"🔐 Your TOTP Secret: {secret}")


#### Step 2: Generate a Provisioning URI for QR Code

In [ ]:
# Replace these values with your actual service and user
totp = pyotp.TOTP(secret)
provisioning_uri = totp.provisioning_uri(name="aatiz.ghimire@heraldcollege.edu.np", issuer_name="SDE-CyberSecurity101")
print("🧾 Provisioning URI:", provisioning_uri)

#### Step 3: Create QR Code for Google Authenticator

In [ ]:
import qrcode
from PIL import Image

# Generate and display QR code
img = qrcode.make(provisioning_uri)
img.save("totp_qr.png")

# Display the QR code image
Image.open("totp_qr.png")


**👉 Scan this QR code with your Google Authenticator app. You will begin to see 6-digit TOTP codes that update every 30 seconds.**

####Step 4: Verify the TOTP Code

In [ ]:
user_input = input("Enter the 6-digit code from Google Authenticator: ")
if totp.verify(user_input):
    print("✅ TOTP Verified Successfully!")
else:
    print("❌ Invalid TOTP!")


#### 📌 Optional Extension

In [ ]:
# Print current OTP (server simulation)
print("⏳ Current OTP from server side:", totp.now())


### Task 3: Encoding and Decoding Images using Base64

**1. Import Libraries**

In [ ]:
import base64
from google.colab import files
from IPython.display import Image as ColabImage

**2.Upload and Encode an Image**

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

with open(filename, "rb") as image_file:
    encoded = base64.b64encode(image_file.read())
    print("Encoded String (truncated):", encoded[:100])

**3. Decode and Save the Image**

In [ ]:
with open("decoded_" + filename, "wb") as f:
    f.write(base64.b64decode(encoded))

ColabImage("decoded_" + filename)

**Verify that the decoded image matches the uploaded one visually.**

### Task 4: Tokenizing Sensitive Fields in JSON



#### Step 1: Define user data

In [ ]:
import json, uuid

user_data = {
    "name": "aatiz",
    "email": "aatiz.ghimire@heraldcollege.edu.np",
    "credit_card": "4111-1111-1111-1111",
    "ssn": "1234",
    "expiration_date": "12/24"
}


#### Step 2: Tokenize JSON

In [ ]:
# Token vault
token_map = {}

def tokenize_value(value):
    token = str(uuid.uuid4())
    token_map[token] = value
    return token


In [ ]:
# Tokenize sensitive fields
for key in ["credit_card", "ssn"]:
    user_data[key] = tokenize_value(user_data[key])

print("Tokenized JSON:", json.dumps(user_data, indent=2))
print("Vault:", json.dumps(token_map, indent=2))

#### Step3: De-Tokenize JSON

In [ ]:
# Reconstruct original data
for token, value in token_map.items():
    for key, val in user_data.items():
        if val == token:
            user_data[key] = value

print("Restored JSON:", json.dumps(user_data, indent=2))

## End of Workshop

Congratulations! You have implemented secure password storage, TOTP-based MFA, safe file encoding, and tokenized JSON data using Python tools.

